# CHAPTER 8 - EXERTCISE 3

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

In [3]:
sc= SparkContext(master= 'local', appName= 'Chapter 8 - Exercise 3')
ss= SparkSession(sc)

## Nhập dữ liệu

In [69]:
path_data= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Chapter6/kdd_data/kddcup.data_10_percent.gz'
df= ss.read.csv(path, inferSchema= True)

## Kiểm tra dữ liệu

In [70]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: d

In [71]:
from pyspark.sql.functions import count, when, isnan, isnull, col
nan_data= df.select([count(when(isnan(c), c)).alias(c + '_nan') for c in df.columns]).toPandas().T
nan_data

,0
_c0_nan,0
_c1_nan,0
_c2_nan,0
_c3_nan,0
_c4_nan,0
_c5_nan,0
_c6_nan,0
_c7_nan,0
_c8_nan,0
_c9_nan,0


In [72]:
null_data= df.select([count(when(isnull(c), c)).alias(c + '_null') for c in df.columns]).toPandas().T
null_data

,0
_c0_null,0
_c1_null,0
_c2_null,0
_c3_null,0
_c4_null,0
_c5_null,0
_c6_null,0
_c7_null,0
_c8_null,0
_c9_null,0


## Tạo tập train, test

In [73]:
train, test= df.randomSplit([0.8, 0.2])

## Tạo pipeline xử lý dữ liệu

In [74]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [75]:
_c1_str_indexer= StringIndexer(inputCol= '_c1', outputCol= '_c1_idx')
_c2_str_indexer= StringIndexer(inputCol= '_c2', outputCol= '_c2_idx')
_c3_str_indexer= StringIndexer(inputCol= '_c3', outputCol= '_c3_idx')
label_str_indexer= StringIndexer(inputCol= '_c41', outputCol= 'label')

In [78]:
vector_assembler= VectorAssembler(inputCols= ['_c0', '_c1_idx', '_c2_idx', '_c3_idx', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23', '_c24', '_c25', '_c26', '_c27', '_c28', '_c29', '_c30', '_c31', '_c32', '_c33', '_c34', '_c35', '_c36', '_c37', '_c38', '_c39', '_c40'],
                                  outputCol= 'features')

In [81]:
process= Pipeline(stages= [_c1_str_indexer, _c2_str_indexer, _c3_str_indexer, label_str_indexer, vector_assembler])
process_model= process.fit(train)

## Tạo model

In [80]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [82]:
train_cleaned= process_model.transform(train)

In [87]:
# DecisionTreeClassifier
dt_classifier= DecisionTreeClassifier(featuresCol= 'features', labelCol= 'label', maxBins= 70)
dt_classifier_model= dt_classifier.fit(train_cleaned)
dt_train_result= dt_classifier_model.transform(train_cleaned)

In [91]:
# RandomForestClassifier
rf_classifier= RandomForestClassifier(featuresCol= 'features', labelCol= 'label', maxBins= 70)
rf_classifier_model= rf_classifier.fit(train_cleaned)
rf_train_result= dt_classifier_model.transform(train_cleaned)

## Đánh giá mô hình

In [124]:
def evaluator(data_result):
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
    mutil_classification_evaluator= MulticlassClassificationEvaluator(labelCol= 'label')
    accuracy= mutil_classification_evaluator.setMetricName('accuracy')
    f1= mutil_classification_evaluator.setMetricName('f1')
    bi_classification_evaluator= BinaryClassificationEvaluator(labelCol= 'label')
    auc= bi_classification_evaluator.setMetricName('areaUnderROC')
    print('Accuracy:', accuracy.evaluate(data_result))
    print('F1:', f1.evaluate(data_result))
    print('AUC:', auc.evaluate(data_result))

### Tập train

In [117]:
evaluator(dt_train_result)

Accuracy: 0.9850291315163052
F1: 0.9850291315163052
AUC: 0.9900528356432942
Confusion matrix:


In [118]:
evaluator(rf_train_result)

Accuracy: 0.9850291315163052
F1: 0.9850291315163052
AUC: 0.9900528356432942
Confusion matrix:


### Tập test

In [119]:
test_cleaned= process_model.transform(test)

In [120]:
dt_test_result= dt_classifier_model.transform(test_cleaned)

In [121]:
evaluator(dt_test_result)

Accuracy: 0.984320259600271
F1: 0.984320259600271
AUC: 0.9907017560113821
Confusion matrix:


In [122]:
rf_test_result= dt_classifier_model.transform(test_cleaned)

In [123]:
evaluator(rf_test_result)

Accuracy: 0.984320259600271
F1: 0.984320259600271
AUC: 0.9907017560113821
Confusion matrix:
